# Q&A
* how to determine data split between train, validation and test sets? Since we cannot use the test set for any observation, is it even possible?  
* is there conv3d, to use for the viedo data?
* maxpool vs avgpool?
* why big filters do not improve the performance by much?
* how to manage memory correctly? because i think that my current code does generate some trash, that is noe beeing removes (as from time to time i use all available memory, but it appears to be rather random)

# Setup

## Libraries

In [1]:
%matplotlib inline

In [2]:
!pip install matplotlib torch torchvision numpy pandas scikit-learn wandb

In [3]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import nn

import wandb

from io import StringIO
import sys

## Config

In [ ]:
batch_size = 64

In [4]:
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {_device}")

Using device: cpu


In [5]:
data_augmentation = True

### Helpers


In [ ]:
# https://stackoverflow.com/questions/16571150/how-to-capture-stdout-output-from-a-python-function-call

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout


## Import data
**About data:** The dataset consists of 102 flower categories, and each class has between 40 and 258 images. The images have large scale, pose, and light variations. In addition, there are categories that have large variations within the category and several very similar categories.  
The default split of the dataset is 1020, 1020 and 6149 images for training, validation and test sets respectively.
If you can handle the bigger training dataset, you can experiment by taking up to 80% of the test set for training.

In [6]:
class InMemDataLoader(object):
    """
    A data loader that keeps all data in CPU or GPU memory.
    """

    __initialized = False

    def __init__(
        self,
        dataset,
        batch_size=1,
        shuffle=False,
        sampler=None,
        batch_sampler=None,
        drop_last=False,
    ):
        """A torch dataloader that fetches data from memory."""
        batches = []
        for i in tqdm(range(len(dataset))):
            batch = [torch.tensor(t) for t in dataset[i]]
            batches.append(batch)
        tensors = [torch.stack(ts) for ts in zip(*batches)]
        dataset = torch.utils.data.TensorDataset(*tensors)
        self.dataset = dataset
        self.batch_size = batch_size
        self.drop_last = drop_last

        if batch_sampler is not None:
            if batch_size > 1 or shuffle or sampler is not None or drop_last:
                raise ValueError(
                    "batch_sampler option is mutually exclusive "
                    "with batch_size, shuffle, sampler, and "
                    "drop_last"
                )
            self.batch_size = None
            self.drop_last = None

        if sampler is not None and shuffle:
            raise ValueError("sampler option is mutually exclusive with " "shuffle")

        if batch_sampler is None:
            if sampler is None:
                if shuffle:
                    sampler = torch.utils.data.RandomSampler(dataset)
                else:
                    sampler = torch.utils.data.SequentialSampler(dataset)
            batch_sampler = torch.utils.data.BatchSampler(
                sampler, batch_size, drop_last
            )

        self.sampler = sampler
        self.batch_sampler = batch_sampler
        self.__initialized = True

    def __setattr__(self, attr, val):
        if self.__initialized and attr in ("batch_size", "sampler", "drop_last"):
            raise ValueError(
                "{} attribute should not be set after {} is "
                "initialized".format(attr, self.__class__.__name__)
            )

        super(InMemDataLoader, self).__setattr__(attr, val)

    def __iter__(self):
        for batch_indices in self.batch_sampler:
            yield self.dataset[batch_indices]

    def __len__(self):
        return len(self.batch_sampler)

    def to(self, device):
        self.dataset.tensors = tuple(t.to(device) for t in self.dataset.tensors)
        return self

In [7]:
def load_flowers(
    batch_size=64,
    test_train_valid_percent=(0.1, 0.8, 0.1),
    train_transform=None,
    eval_transform=None,
    Loader=torch.utils.data.DataLoader,
):
    """
    Load the flowers dataset with the given batch size and transformation.
    The dataset is split into train, validation, and test sets according to the specified percentages.
    The data is loaded using the specified loader class.
    """

    if train_transform is None:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),

            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomRotation(15),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAdjustSharpness(sharpness_factor=2),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
    if eval_transform is None:
        eval_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])


    test_percent, train_percent, valid_percent = test_train_valid_percent

    # TRAIN
    train = torchvision.datasets.Flowers102(
        root='./data', split='train', download=True,
        transform=train_transform if data_augmentation else eval_transform
    )
    train = torch.utils.data.Subset(train, range(int(len(train) * train_percent)))

    # TEST
    test = torchvision.datasets.Flowers102(
        root='./data', split='test', download=True, transform=eval_transform
    )
    test = torch.utils.data.Subset(test, range(int(len(test) * test_percent)))

    # VALID
    valid = torchvision.datasets.Flowers102(
        root='./data', split='val', download=True, transform=eval_transform
    )
    valid = torch.utils.data.Subset(valid, range(int(len(valid) * valid_percent)))

    data_loaders = {
        'train': Loader(train, batch_size=batch_size, shuffle=True),
        'valid': Loader(valid, batch_size=batch_size, shuffle=True),
        'test': Loader(test, batch_size=batch_size, shuffle=True),
    }

    return data_loaders


# Solution

## Task 1
* Your task is to implement a convolutional neural network from scratch using PyTorch.
* Your CNN should consist of convolutional layers (Conv2D), pooling layers (MaxPooling2D), activation layers (e.g., ReLU), and fully connected layers (if needed).

### Import data

In [ ]:
data_loaders = load_flowers(batch_size, (1, 1, 1), Loader=InMemDataLoader)

### Model class

In [ ]:
class Model1(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model1, self).__init__()
        self.layers = nn.Sequential(*args, **kwargs)

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
def evaluate(model, data_loader):
    loss = 0
    correct = 0
    loss_fn = nn.CrossEntropyLoss(
        reduction='sum',
    )

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(_device), labels.to(_device)

            outputs = model(inputs)
            loss += loss_fn(outputs, labels).item()
            pred = outputs.argmax(
                dim=1, keepdim=True
            )
            correct += (
                pred.eq(labels.view_as(pred)).sum().item()
            )

    loss /= len(data_loader.dataset)
    accuracy = correct / len(data_loader.dataset)
    return loss, accuracy

In [ ]:
def train_step(model, optimizer, loss_fn):
    for batch_idx, (inputs, labels) in enumerate(data_loaders['train']):
        inputs, labels = inputs.to(_device), labels.to(_device)

        optimizer.zero_grad()   # Zero gradients
        logits = model(inputs)   # Forward pass
        loss = loss_fn(logits, labels)  # Compute loss
        loss.backward() # Backward pass
        optimizer.step()    # Update weights

        wandb.log({
            "loss": loss.item(),
            "batch_idx": batch_idx,
        })

In [ ]:
def init_weights(model):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
            if layer.bias is not None:
                nn.init.zeros_(layer.bias)
        elif isinstance(layer, nn.BatchNorm2d):
            nn.init.ones_(layer.weight)
            if layer.bias is not None:
                nn.init.zeros_(layer.bias)

### Model creation

In [ ]:
# version 1 of model - too simple
model_type = "v1"

model = Model1(
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(p=0.25),

    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(p=0.25),

    nn.Flatten(),
    nn.Linear(128 * 56 * 56, 512),
    nn.ReLU(),

    nn.Dropout(p=0.5),
    nn.Linear(512, 102),
)

In [ ]:
# version 2
model_type = "v2"

model = Model1(
    nn.Conv2d(3, 128, kernel_size=11, stride=4, padding=2),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=5, stride=2),
    nn.Dropout(p=0.25),

    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(p=0.25),

    nn.Conv2d(256, 256, kernel_size=3),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(p=0.25),

    nn.Flatten(),
    nn.Linear(6400, 512),
    nn.ReLU(),

    nn.Dropout(p=0.5),
    nn.Linear(512, 102),
)

In [ ]:
# stacked 3x3 convs
model_type = "v3_stack3x3"

model = Model1(
    nn.Conv2d(3, 64, kernel_size=5, stride=3, padding=2),
    nn.BatchNorm2d(64),
    nn.ReLU(),

    nn.Conv2d(64, 128, kernel_size=3),
    nn.BatchNorm2d(128),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=3, stride=3),
    nn.Dropout(p=0.25),

    nn.Conv2d(128, 256, kernel_size=3, stride=2),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.Conv2d(256, 256, kernel_size=3),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=3, stride=3),
    nn.Dropout(p=0.25),

    nn.Flatten(),
    nn.Linear(2304, 1024),
    nn.ReLU(),

    nn.Linear(1024, 512),
    nn.ReLU(),

    nn.Dropout(p=0.5),
    nn.Linear(512, 102),
)

In [ ]:
# version 3 = alexnet
model_type = "alexnet"

model = Model1(
    nn.Conv2d(3, 96, kernel_size=11, stride=4),
    nn.BatchNorm2d(96),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(p=0.25),

    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(p=0.25),

    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.BatchNorm2d(384),
    nn.ReLU(),
    nn.Dropout(p=0.25),

    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.BatchNorm2d(384),
    nn.ReLU(),
    nn.Dropout(p=0.25),

    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(p=0.25),


    nn.Flatten(),
    nn.Linear(6400, 4096),
    nn.ReLU(),

    nn.Linear(4096, 4096),
    nn.ReLU(),

    nn.Dropout(p=0.5),
    nn.Linear(4096, 102),
    # nn.Softmax(dim=1),
)

### Training loop

In [ ]:
model.to(_device)

epochs = 100

learning_rate = 0.0001
momentum = 0.9
weight_decay = 0.0005
betas = (0.9, 0.999)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=betas,
    weight_decay=weight_decay,
)
# optimizer = torch.optim.SGD(
#     model.parameters(),
#     lr=learning_rate,
#     momentum=momentum,
#     weight_decay=weight_decay,
# )

loss_fn = nn.CrossEntropyLoss()

init_weights(model)

run = wandb.init(
    entity = "fejowo5522-",
    project= "NN_list3_OxFlow",
    config = {
        "task": 1,
        "batch_size": batch_size,
        "epochs": epochs,
        "optimizer": "SGD",
        "learning_rate": learning_rate,
        # "momentum": momentum,
        "betas": betas,
        "weight_decay": weight_decay,
        "loss_fn": "cross_entropy",
        "model": model_type,
        "data_augmentation": data_augmentation,
    }
)
run.name = "Task1_" + str(int(time.time()))

model.train()
for epoch in tqdm(range(epochs), desc="Training", leave=False):
    train_step(model, optimizer, loss_fn)

    for loader, split in [
        (data_loaders['train'], 'train'),
        (data_loaders['valid'], 'valid'),
    ]:
        loss, accuracy = evaluate(model, loader)
        wandb.log({
            "epoch": epoch,
            f"{split}_loss": loss,
            f"{split}_accuracy": accuracy,
        })

model.eval()
loss, accuracy = evaluate(model, data_loaders['test'])
wandb.log({
    "test_loss": loss,
    "test_accuracy": accuracy,
})
print(
    "Test set: Average loss: {:8.6f}, Accuracy: ({:4.1f}%)".format(
        loss,
        100.0 * accuracy,
    )
)

run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

## Task 2
* Train your CNN on different training set sized (10%, 20%, 50%, 80%, 100%) and evaluate the performance on the validation set and test set.
    * Report the accuracy and loss on the validation set and test set for each training set size.
* Train your CNN on the full training set plus 20%, 50% and 80% of the test set and evaluate the performance on the validation set and the remaining test set.
    * Report the accuracy and loss on the validation set and remaining test set for each training set size.
* Compare the performance of your CNN on the different training set sizes and analyze the results.


In [ ]:
training_sizes = [0.1, 0.2, 0.5, 0.8, 1, 1.2, 1.5, 1.8]
# training_sizes = [1.2, 1.5, 1.8]

In [ ]:
def _train_step(model, optimizer, loss_fn, data_loader, max_batch_percent=1, reverse_loop=False):
    data_iter = iter(data_loader)

    if reverse_loop:
        data_iter = reversed(list(data_iter))

    for batch_idx, (inputs, labels) in enumerate(data_iter):
        if batch_idx >= max_batch_percent * len(data_loader):
            break

        inputs, labels = inputs.to(_device), labels.to(_device)

        optimizer.zero_grad()   # Zero gradients
        logits = model(inputs)   # Forward pass
        loss = loss_fn(logits, labels)  # Compute loss
        loss.backward() # Backward pass
        optimizer.step()    # Update weights

        wandb.log({
            "loss": loss.item(),
            "batch_idx": batch_idx,
        })


def train_step(model, optimizer, loss_fn, max_batch_percent=1):
    _train_step(model, optimizer, loss_fn, data_loaders['train'], max_batch_percent)

    if max_batch_percent > 1:
        _train_step(model, optimizer, loss_fn, data_loaders['test'], max_batch_percent - 1, True)

In [ ]:
def evaluate(model, data_loader, max_batch_percent=1):
    loss = 0
    correct = 0
    loss_fn = nn.CrossEntropyLoss(
        reduction='sum',
    )

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(data_loader):


            inputs, labels = inputs.to(_device), labels.to(_device)

            outputs = model(inputs)
            loss += loss_fn(outputs, labels).item()
            pred = outputs.argmax(
                dim=1, keepdim=True
            )
            correct += (
                pred.eq(labels.view_as(pred)).sum().item()
            )

    loss /= len(data_loader.dataset)
    accuracy = correct / len(data_loader.dataset)
    return loss, accuracy

In [ ]:
model.to(_device)

epochs = 5

learning_rate = 0.0001
momentum = 0.9
weight_decay = 0.0005
betas = (0.9, 0.999)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=betas,
    weight_decay=weight_decay,
)
# optimizer = torch.optim.SGD(
#     model.parameters(),
#     lr=learning_rate,
#     momentum=momentum,
#     weight_decay=weight_decay,
# )

loss_fn = nn.CrossEntropyLoss()

for training_size in training_sizes:
    print(f"====>   Training size: {training_size}")

    init_weights(model)

    run = wandb.init(
        entity = "fejowo5522-",
        project= "NN_list3_OxFlow",
        config = {
            "task": 2,
            "batch_size": batch_size,
            "epochs": epochs,
            "optimizer": "SGD",
            "learning_rate": learning_rate,
            # "momentum": momentum,
            "betas": betas,
            "weight_decay": weight_decay,
            "loss_fn": "cross_entropy",
            "model": model_type,
            "training_size": training_size,
            "data_augmentation": data_augmentation,
        }
    )
    run.name = "Task2_" + str(int(time.time()))

    model.train()

    for epoch in tqdm(range(epochs), desc="Training", leave=False):
        train_step(model, optimizer, loss_fn, training_size)

        for loader, split in [
            (data_loaders['train'], 'train'),
            (data_loaders['valid'], 'valid'),
        ]:
            loss, accuracy = evaluate(model, loader)
            wandb.log({
                "epoch": epoch,
                f"{split}_loss": loss,
                f"{split}_accuracy": accuracy,
            })

    model.eval()
    loss, accuracy = evaluate(model, data_loaders['test'], 1 if training_size < 1 else 2 - training_size)
    wandb.log({
        "test_loss": loss,
        "test_accuracy": accuracy,
    })
    print(
        "Test set: Average loss: {:8.6f}, Accuracy: ({:4.1f}%)".format(
            loss,
            100.0 * accuracy,
        )
    )

    run.finish()

====>   Training size: 0.1


batch_idx,▄▇▃▁▂▄▅▅█▃▅▄▁▂▃▂▄▃▇▂█▄▇▁▂▇▃▄▅▆▄▆▂▂▃▆▃▄▆▅
epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁
train_loss,▁▆█▅▅▅███▄
valid_accuracy,▁▁▁▁▁▁▁▁▁▁
valid_loss,▁▁▁▂▃▃▄▅▇█
batch_idx,15
epoch,9
loss,4.62315
train_accuracy,0.0098


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Test set: Average loss: 4.619931, Accuracy: ( 1.0%)


batch_idx,▁█▁█▁█▁█▁█
epoch,▁▁▃▃▅▅▆▆██
loss,▁██▆▅▃▂▁▁▁
test_accuracy,▁
test_loss,▁
train_accuracy,▅▂▆█▁
train_loss,█▅▂▁▁
valid_accuracy,▂▃█▁▂
valid_loss,█▅▂▁▁
batch_idx,1
epoch,4


...

batch_idx,▁▂▃▂▂▄▆▆▂▁▂▂▅▅▆█▃▄▄▄▇▃▃▁▁▆▇██▁▃▃▂▂▃▄▅▅▆█
epoch,▁▁▃▃▅▅▆▆██
loss,█▅▃▄▃▃▁▄▃▃▃▃▄▃▃▃▃▃▅▄▃▃▃▃▃▃▃▄▃▃▃▂▃▄▃▄▃▂▃▂
test_accuracy,▁
test_loss,▁
train_accuracy,▁▁▁▁▁
train_loss,▁▄▇██
valid_accuracy,▁▁▁▁▁
valid_loss,▁▂▃▅█
batch_idx,48
epoch,4


====>   Training size: 1.8


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Test set: Average loss: 4.614853, Accuracy: ( 3.9%)


batch_idx,▁▂▁▂▃█▁▂▂▂▄▆▇▇█▂▂▂▁▂▅▅▅▇▇▂▂▃▃▅▂▂▃▄▄▅▆▇▇█
epoch,▁▁▃▃▅▅▆▆██
loss,▅█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
test_loss,▁
train_accuracy,▁▁▁▁▁
train_loss,▁▁▆▇█
valid_accuracy,▁▁▁▁▁
valid_loss,▁▂▃▅█
batch_idx,77
epoch,4


## Task 3
* Implement a baseline AlexNet model using PyTorch.
* Training AlexNet may take a long time, so try to use GPU acceleration if available.


### Model class

In [51]:
class AlexNetBaseline(nn.Module):
    def __init__(self, num_classes=102):
        super(AlexNetBaseline, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def init_weights(self):
        for layer in self.modules():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)
            elif isinstance(layer, nn.BatchNorm2d):
                nn.init.ones_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def evaluate(self, data_loader):
        loss = 0
        correct = 0
        loss_fn = nn.CrossEntropyLoss(
            reduction='sum',
        )

        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(_device), labels.to(_device)

                outputs = self(inputs)
                loss += loss_fn(outputs, labels).item()
                pred = outputs.argmax(
                    dim=1, keepdim=True
                )
                correct += (
                    pred.eq(labels.view_as(pred)).sum().item()
                )

        loss /= len(data_loader.dataset)
        accuracy = correct / len(data_loader.dataset)
        return loss, accuracy

In [52]:
def train_step(model, optimizer, loss_fn, data_loader):
    data_iter = iter(data_loader)

    for batch_idx, (inputs, labels) in enumerate(data_iter):
        inputs, labels = inputs.to(_device), labels.to(_device)

        optimizer.zero_grad()   # Zero gradients
        logits = model(inputs)   # Forward pass
        loss = loss_fn(logits, labels)  # Compute loss
        loss.backward() # Backward pass
        optimizer.step()    # Update weights

        wandb.log({
            "loss": loss.item(),
            "batch_idx": batch_idx,
        })

### Train model

In [ ]:
batch_size = 64
data_loaders = load_flowers(batch_size, (1, 1, 1), transform=None, Loader=torch.utils.data.DataLoader)

In [20]:
# initialize
alexnet_model = AlexNetBaseline(num_classes=102).to(_device)
print(alexnet_model)

AlexNetBaseline(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout

In [22]:
# parameters
epochs = 100
learning_rate = 0.001
weight_decay = 0.0005

# optimizer and loss
optimizer = torch.optim.Adam(alexnet_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

alexnet_model.init_weights()

run = wandb.init(
    entity = "fejowo5522-",
    project= "NN_list3_OxFlow",
    config = {
        "task": 3,
        "batch_size": batch_size,
        "epochs": epochs,
        "optimizer": "Adam",
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "loss_fn": "cross_entropy",
        "model": 'alexnet',
        "data_augmentation": data_augmentation,
    }
)
run.name = "Task3_" + str(int(time.time()))


alexnet_model.train()
for epoch in tqdm(range(epochs), desc="Training", leave=False):
    train_step(alexnet_model, optimizer, loss_fn, data_loaders['train'])

    for loader, split in [
        (data_loaders['train'], 'train'),
        (data_loaders['valid'], 'valid'),
    ]:
        loss, accuracy = alexnet_model.evaluate(loader)
        wandb.log({
            "epoch": epoch,
            f"{split}_loss": loss,
            f"{split}_accuracy": accuracy,
        })

alexnet_model.eval()
loss, accuracy = alexnet_model.evaluate(data_loaders['test'])
wandb.log({
    "test_loss": loss,
    "test_accuracy": accuracy,
})
print(
    "Test set: Average loss: {:8.6f}, Accuracy: ({:4.1f}%)".format(
        loss,
        100.0 * accuracy,
    )
)

run.finish()

Training:   0%|          | 0/100 [00:00<?, ?it/s]

Test set: Average loss: 4.625029, Accuracy: ( 0.7%)


batch_idx,▆▁▃▅▄▇▇█▇█▃▄▁▄▅▁▆▁▆█▃▃▇▇▃▇▂▅█▆▄▅▁▃▃▂█▅▄█
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█
loss,█▁▇▄▃▃▂▄▄▃▂▄▃▄▃▂▃▃▃▃▃▃▂▂▃▃▃▃▄▂▃▄▃▃▂▃▄▂▃▂
test_accuracy,▁
test_loss,▁
train_accuracy,█▅▁▅█▄▅▅▅▅▅▅▅▅▄▅▅▅▅▅▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
train_loss,█▆▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_accuracy,█▇▇▆▇▁█▆▆▇█▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
valid_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_idx,15
epoch,99


## Task 4
* Input normalization: experiment with different input normalization techniques (e.g., mean subtraction, standardization) and analyze their impact on the model's performance.


In [ ]:
def train_step(model, optimizer, loss_fn, data_loader):
    data_iter = iter(data_loader)

    for batch_idx, (inputs, labels) in enumerate(data_iter):
        inputs, labels = inputs.to(_device), labels.to(_device)

        optimizer.zero_grad()   # Zero gradients
        logits = model(inputs)   # Forward pass
        loss = loss_fn(logits, labels)  # Compute loss
        loss.backward() # Backward pass
        optimizer.step()    # Update weights

        wandb.log({
            "loss": loss.item(),
            "batch_idx": batch_idx,
        })

In [65]:
def test_data_loaders(
    normalization_method,
    data_loaders,
    model,
    epochs=10,
    learning_rate=0.001,
    weight_decay = 0.0005,
):
    # optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    loss_fn = nn.CrossEntropyLoss()

    model.init_weights()

    run = wandb.init(
        entity = "fejowo5522-",
        project= "NN_list3_OxFlow",
        config = {
            "task": 4,
            "batch_size": batch_size,
            "epochs": epochs,
            "optimizer": "Adam",
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "loss_fn": "cross_entropy",
            "model": 'alexnet',
            "normalization": normalization_method,
            "data_augmentation": data_augmentation,
        }
    )
    run.name = "Task4_" + str(int(time.time()))


    model.train()
    for epoch in tqdm(range(epochs), desc="Training", leave=False):
        train_step(model, optimizer, loss_fn, data_loaders['train'])

        for loader, split in [
            (data_loaders['train'], 'train'),
            (data_loaders['valid'], 'valid'),
        ]:
            loss, accuracy = model.evaluate(loader)
            wandb.log({
                "epoch": epoch,
                f"{split}_loss": loss,
                f"{split}_accuracy": accuracy,
            })

    model.eval()
    loss, accuracy = model.evaluate(data_loaders['test'])
    wandb.log({
        "test_loss": loss,
        "test_accuracy": accuracy,
    })
    print(
        "Test set: Average loss: {:8.6f}, Accuracy: ({:4.1f}%)".format(
            loss,
            100.0 * accuracy,
        )
    )

    run.finish()

In [66]:
normalization_transforms = {
    'mean_subtraction': transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[1, 1, 1]),
    'standardization': transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
}

for name, normalization in normalization_transforms.items():
    data_loaders = load_flowers(
        batch_size,
        (1,1,1),
        train_transform=transforms.Compose([
            transforms.Resize((224, 224)),

            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomRotation(15),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAdjustSharpness(sharpness_factor=2),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

            transforms.ToTensor(),
            normalization,
        ]),
        eval_transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalization,
        ]),
        Loader=torch.utils.data.DataLoader
    )


    model = AlexNetBaseline(num_classes=102).to(_device)

    test_data_loaders(name, data_loaders, model)



Training:   0%|          | 0/10 [00:00<?, ?it/s]

Test set: Average loss: 4.135406, Accuracy: ( 3.7%)


batch_idx,▂▃▄▄▁▄▅█▇█▂▃▃▄▅▂▃▅██▄▄▅▂▂▄▅▇█▁▃▆█▅▇█▂▃▄█
epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
loss,██████████▇▇▅▆▇█▇▇▆▇▇▅▆▅▅▅▆▅▆▆▅▅▄▃▃▂▃▁▁▃
test_accuracy,▁
test_loss,▁
train_accuracy,▁▁▂▃▃▃▄▆▆█
train_loss,██▇▇▇▆▆▄▂▁
valid_accuracy,▁▂▁▃▃▅▅█▇█
valid_loss,██▇▇▇▆▅▄▂▁
batch_idx,15
epoch,9


Training:   0%|          | 0/10 [00:00<?, ?it/s]

Test set: Average loss: 4.626478, Accuracy: ( 1.1%)


batch_idx,▁▂▃▄█▅▂▆▇█▁▁▂▆▆▁▂▃▆▇▂▂▆█▁▅▇█▁▄▁▁▂▅▆▁▂▂▄█
epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
test_loss,▁
train_accuracy,▆▆▆█▆▆▆▆▆▁
train_loss,██▄▃▃▂▁▂▁▂
valid_accuracy,▅█▅▅▅▇▂▂▅▁
valid_loss,▆█▃▃▃▂▂▂▁▁
batch_idx,15
epoch,9


## Zad 5
* Experiment with different hyperparameters such as learning rate, batch size, number of epochs, and optimizer choice (e.g., SGD, Adam).

### Testing function

In [9]:
def random_search_tuner(
    param_ranges,
    model_name,
    model_class,
    optimizer_name='Adam',
    loss_fn_name='cross_entropy',
    trials=10,
    epochs=10,
):
    optimizer = {
        'Adam': torch.optim.Adam,
        'SGD': torch.optim.SGD,
    }[optimizer_name]

    loss_fn = {
        'cross_entropy': nn.CrossEntropyLoss,
        'mse': nn.MSELoss,
    }[loss_fn_name]
    loss_fn = loss_fn()

    for trial in tqdm(range(trials), desc="Trial", leave=False, position=0):
        # randomly sample hyperparameters
        variables = {
            'learning_rate': 0,
            'weight_decay': 0,
            'momentum': 0,
            'batch_size': 0,
        }

        for name, variable in variables.items():
            if name in param_ranges:
                variables[name] = np.random.uniform(param_ranges[name][0], param_ranges[name][1])

        # init model
        model = model_class().to(_device)
        model.init_weights()

        # create optimizer
        if(optimizer_name == 'SGD'):
            _optimizer = optimizer(
                model.parameters(),
                lr=variables['learning_rate'],
                weight_decay=variables['weight_decay'],
                momentum=variables['momentum'],
            )
        elif(optimizer_name == 'Adam'):
            _optimizer = optimizer(
                model.parameters(),
                lr=variables['learning_rate'],
                weight_decay=variables['weight_decay'],
            )

        # create data loaders
        data_percent = (1,1,1)
        data_loaders = load_flowers(
            batch_size,
            data_percent,
        )

        # init wandb
        with Capturing() as output:
            run = wandb.init(
                entity = "fejowo5522-",
                project= "NN_list3_OxFlow",
                config = {
                    "task": 5,
                    "batch_size": variables['batch_size'],
                    "epochs": epochs,
                    "optimizer": optimizer_name,
                    "learning_rate": variables['learning_rate'],
                    "momentum": variables['momentum'],
                    "weight_decay": variables['weight_decay'],
                    "loss_fn": loss_fn_name,
                    "model": model_name,
                    "data_percent": data_percent,
                    "data_augmentation": data_augmentation,
                }
            )
            run.name = "Task5_" + str(int(time.time()))

        # train model
        model.train()
        for epoch in tqdm(range(epochs), desc="Epoch", leave=False, position=1):
            train_step(model, _optimizer, loss_fn, data_loaders['train'])

            for loader, split in [
                (data_loaders['train'], 'train'),
                (data_loaders['valid'], 'valid'),
            ]:
                loss, accuracy = model.evaluate(loader)
                wandb.log({
                    "epoch": epoch,
                    f"{split}_loss": loss,
                    f"{split}_accuracy": accuracy,
                })

        model.eval()
        loss, accuracy = model.evaluate(data_loaders['test'])
        wandb.log({
            "test_loss": loss,
            "test_accuracy": accuracy,
        })
        print(
            "Test set: Average loss: {:8.6f}, Accuracy: ({:4.1f}%)".format(
                loss,
                100.0 * accuracy,
            )
        )

        run.finish()


### Model class

In [10]:
def train_step(model, optimizer, loss_fn, data_loader):
    data_iter = iter(data_loader)

    for batch_idx, (inputs, labels) in enumerate(data_iter):
        inputs, labels = inputs.to(_device), labels.to(_device)

        optimizer.zero_grad()   # Zero gradients
        logits = model(inputs)   # Forward pass
        loss = loss_fn(logits, labels)  # Compute loss
        loss.backward() # Backward pass
        optimizer.step()    # Update weights

        wandb.log({
            "loss": loss.item(),
            "batch_idx": batch_idx,
        })

In [11]:
class AlexNetBaseline(nn.Module):
    def __init__(self, num_classes=102):
        super(AlexNetBaseline, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def init_weights(self):
        for layer in self.modules():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)
            elif isinstance(layer, nn.BatchNorm2d):
                nn.init.ones_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def evaluate(self, data_loader):
        loss = 0
        correct = 0
        loss_fn = nn.CrossEntropyLoss(
            reduction='sum',
        )

        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(_device), labels.to(_device)

                outputs = self(inputs)
                loss += loss_fn(outputs, labels).item()
                pred = outputs.argmax(
                    dim=1, keepdim=True
                )
                correct += (
                    pred.eq(labels.view_as(pred)).sum().item()
                )

        loss /= len(data_loader.dataset)
        accuracy = correct / len(data_loader.dataset)
        return loss, accuracy

### Test parameters

In [12]:
random_search_tuner(
    param_ranges={
        'learning_rate': (1e-5, 1e-1),
        'weight_decay': (0.0, 0.1),
    },
    model_name = 'AlexNetBaseline',
    model_class = AlexNetBaseline,
    optimizer_name='Adam',
    loss_fn_name='cross_entropy',
    trials=10,
    epochs=10,
)

Trial:   0%|          | 0/10 [00:00<?, ?it/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fejowo5522 (fejowo5522-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
random_search_tuner(
    param_ranges={
        'learning_rate': (1e-5, 1e-1),
        'weight_decay': (0.0, 0.1),
        'momentum': (0.0, 0.01),
    },
    model_name = 'AlexNetBaseline',
    model_class = AlexNetBaseline,
    optimizer_name='SGD',
    loss_fn_name='cross_entropy',
    trials=10,
    epochs=20,
)

## Zad 6
* Modify your CNN architecture to include batch normalization and dropout layers.
* Experiment with different dropout rates and analyze their impact on the model's performance.


### Model class

In [13]:
class AlexNetBaseline(nn.Module):
    def __init__(self, dropout_p=0.5, num_classes=102):
        super(AlexNetBaseline, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.Dropout(p=dropout_p),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.Dropout(p=dropout_p),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.Dropout(p=dropout_p),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.Dropout(p=dropout_p),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.Dropout(p=dropout_p),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_p),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(p=dropout_p),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def init_weights(self):
        for layer in self.modules():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)
            elif isinstance(layer, nn.BatchNorm2d):
                nn.init.ones_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def evaluate(self, data_loader):
        loss = 0
        correct = 0
        loss_fn = nn.CrossEntropyLoss(
            reduction='sum',
        )

        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(_device), labels.to(_device)

                outputs = self(inputs)
                loss += loss_fn(outputs, labels).item()
                pred = outputs.argmax(
                    dim=1, keepdim=True
                )
                correct += (
                    pred.eq(labels.view_as(pred)).sum().item()
                )

        loss /= len(data_loader.dataset)
        accuracy = correct / len(data_loader.dataset)
        return loss, accuracy

### Training functions

In [14]:
def train_step(model, optimizer, loss_fn, data_loader):
    data_iter = iter(data_loader)

    for batch_idx, (inputs, labels) in enumerate(data_iter):
        inputs, labels = inputs.to(_device), labels.to(_device)

        optimizer.zero_grad()   # Zero gradients
        logits = model(inputs)   # Forward pass
        loss = loss_fn(logits, labels)  # Compute loss
        loss.backward() # Backward pass
        optimizer.step()    # Update weights

        wandb.log({
            "loss": loss.item(),
            "batch_idx": batch_idx,
        })

In [15]:
def test_dropout(
    dropout,
    model_class,
    epochs=10,
    learning_rate=0.001,
    weight_decay = 0.0005,
):
    model = model_class(dropout)
    model.init_weights()

    # optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    loss_fn = nn.CrossEntropyLoss()

    with Capturing() as output:
        run = wandb.init(
            entity = "fejowo5522-",
            project= "NN_list3_OxFlow",
            config = {
                "task": 6,
                "batch_size": batch_size,
                "epochs": epochs,
                "data_augmentation": data_augmentation,
                "optimizer": "Adam",
                "loss_fn": "cross_entropy",
                "learning_rate": learning_rate,
                "weight_decay": weight_decay,
                "model": 'alexnet',
                "dropout": dropout,
            }
        )
        run.name = "Task6_" + str(int(time.time()))


    model.train()
    for epoch in tqdm(range(epochs), desc="Training", leave=False, position=1):
        train_step(model, optimizer, loss_fn, data_loaders['train'])

        for loader, split in [
            (data_loaders['train'], 'train'),
            (data_loaders['valid'], 'valid'),
        ]:
            loss, accuracy = model.evaluate(loader)
            wandb.log({
                "epoch": epoch,
                f"{split}_loss": loss,
                f"{split}_accuracy": accuracy,
            })

    model.eval()
    loss, accuracy = model.evaluate(data_loaders['test'])
    wandb.log({
        "test_loss": loss,
        "test_accuracy": accuracy,
    })
    print(
        "Test set: Average loss: {:8.6f}, Accuracy: ({:4.1f}%)".format(
            loss,
            100.0 * accuracy,
        )
    )

    run.finish()

### Test dropouts

In [ ]:
for drop in tqdm(range(0, 100, 10), position=0):
    test_dropout(
        drop/100,
        AlexNetBaseline,
        epochs=10,
        learning_rate=0.001,
        weight_decay = 0.0005,
    )

batch_idx,▁
loss,▁
batch_idx,0
loss,5.8069


Training:   0%|          | 0/10 [00:00<?, ?it/s]

## Zad 7
* Implement data augmentation techniques such as random rotations, shifts, flips, and zooms on the training dataset.
* Train your CNN with augmented data and compare the performance with the baseline model trained on the original data.


implemented already in `load_flowers` data loader and global `data_augmentation` variable or custom `transfrom`'s

## Zad 8
* Implement residual connections in your CNN architecture; see the [ResNet paper](https://arxiv.org/abs/1512.03385) for more details.
* Implement inception modules in your CNN architecture; see the [GoogLeNet paper](https://arxiv.org/abs/1409.4842) for more details.
            